In [155]:
import pandas as pd

In [156]:
up_to_2021 = pd.read_csv("/kaggle/input/monthly-gold-prices/1979-2021.csv")

up_to_2021 = up_to_2021[["Date", "Egypt(EGP)"]].dropna().reset_index(drop=True)

up_to_2021["Egypt(EGP)"] = up_to_2021["Egypt(EGP)"] / 31.1035

up_to_2021 = up_to_2021.rename(columns={"Egypt(EGP)": "24k"})

In [157]:
up_to_2021['Date'] = pd.to_datetime(up_to_2021['Date'], dayfirst=True)

up_to_2021['Date'] = up_to_2021['Date'].dt.to_period('Q').astype(str)

up_to_2021 = up_to_2021.groupby('Date').mean(numeric_only=True).reset_index()

In [158]:
usd = pd.read_csv("/kaggle/input/usd-to-egp/USD_EGP Historical Data1.csv")

usd = usd[["Date", "Price"]]

usd['Date'] = pd.to_datetime(usd['Date'])

usd['Date'] = usd['Date'].dt.to_period('Q').astype(str)

usd = usd.groupby('Date').mean(numeric_only=True).reset_index()

In [159]:
from_2022 = pd.read_csv("/kaggle/input/egypt-gold-prices-daily-updated/data.csv")

from_2022['24k'] = (from_2022["24K - Local Price/Sell"] + from_2022["24K - Local Price/Buy"]) / 2

from_2022 = from_2022[["Date", "24k"]]
from_2022 = from_2022.iloc[:-25]

from_2022['Date'] = pd.to_datetime(from_2022['Date'])

from_2022['Date'] = from_2022['Date'].dt.to_period('Q').astype(str)

from_2022 = from_2022.groupby('Date').mean(numeric_only=True).reset_index()

In [160]:
data = pd.concat([up_to_2021, from_2022], ignore_index=True)

In [161]:
!pip install requests beautifulsoup4 pandas

In [186]:
import requests
from bs4 import BeautifulSoup

# URL
url = "https://www.exchange-rates.org/precious-metals/gold-price/egypt/2025"

# Send GET request with headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
response = requests.get(url, headers=headers)

rows = []
# Check if request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all <tr> elements with class 'row botton month'
    all_data = soup.find_all('tr', class_='month-row-bottom')
    
    for row in all_data:
        rows.append(row.text.strip())  # Or parse specific <td> elements as needed
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")

In [187]:
import re
from datetime import datetime

# Regex pattern to extract the values
pattern = r"(Lowest|Highest|Average) gold price in Egypt in (\w+) (\d{4}): EGP ([\d,]+)"

# Conversion factor
OUNCE_TO_GRAM = 31.1035

# Dictionary to store results
results = {}

for text in rows:
    for match in re.finditer(pattern, text):
        label, month_str, year_str, price_str = match.groups()
        price = float(price_str.replace(",", ""))
        
        # Convert price to gram
        price_per_gram = round(price / OUNCE_TO_GRAM, 2)
        
        # Standardize date format: MM-YYYY
        month_num = datetime.strptime(month_str, "%B").month
        date_key = f"{month_num:02d}-{year_str}"
        
        # Store result
        if date_key not in results:
            results[date_key] = {}
        results[date_key][label.lower()] = price_per_gram

In [188]:
results = pd.DataFrame(results).T.reset_index()
results = results.rename(columns={"index": "Date"})

In [165]:
def to_quarter_key(month_year):
    month, year = map(int, month_year.split('-'))
    quarter = (month - 1) // 3 + 1
    return f"{year}Q{quarter}"

In [189]:
results["Date"] = results["Date"].apply(to_quarter_key)
results = results[["Date", "average"]]
results = results.groupby('Date')['average'].mean().reset_index()
results = results.rename(columns={"average": "24k"})

In [190]:
data = pd.concat([data, results], ignore_index=True)

In [191]:
data = data.drop_duplicates(subset=["Date"]).reset_index(drop=True)
data = data.sort_values(by='Date').reset_index(drop = True)

In [193]:
data.to_csv("gold_prices_1998_2025_24k.csv")

In [195]:
data["21k"] = data["24k"] * (21 / 24)
data["18k"] = data["24k"] * (18 / 24)

In [197]:
data.to_csv("gold_prices_1998_2025_24k_21k_18k.csv")